<a href="https://colab.research.google.com/github/a-m-kacz/MLA/blob/main/lab4_regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reguralization in CNN networks
- we are going to compare various approaches to regularizarion in neural networks,
- we will use standard image classification task with CNN networks.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import numpy as np

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})

model_args = {}
# we will use batch size of 64 in Stochastic Gradient Descent (SGD) optimization of the network
model_args['batch_size'] = 64 
# learning rate is how fast it will descend
model_args['lr'] = .07
# the number of epochs is the number of times you go through the full dataset
model_args['epochs'] = 30
# L2 (ridge) penalty
model_args['L2_lambda'] = 5.e-3
# L1 (LASSO) penalty
model_args['L1_lambda'] = 2.e-4

### Define problem
- we are going to use CIFAR-10 [dataset](https://www.cs.toronto.edu/~kriz/cifar.html) of 32x32 color images to train image classifier,
- in order to test various regularizations our vanilla CNN classifier should be overfitted.

### Download the dataset

In [2]:
!rm -r ./data
# normalize dataset
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean, std)])

cifar10_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
# we divide this data into training and validation subsets
train_subset, validation_subset = torch.utils.data.random_split(cifar10_train, [40000, 10000])
test_subset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# subsample to speedup training (colab has notebook lifetime limit)
train_subset = torch.utils.data.Subset(train_subset, range(20000))
validation_subset = torch.utils.data.Subset(validation_subset, range(5000))
test_subset = torch.utils.data.Subset(test_subset, range(5000))

# define dataloaders
loader_kwargs = {'batch_size': model_args['batch_size'], 
                 'num_workers': 2, 
                 'pin_memory': True, 
                 'shuffle': True}
train_loader = torch.utils.data.DataLoader(train_subset, **loader_kwargs)
validation_loader = torch.utils.data.DataLoader(validation_subset, **loader_kwargs)
test_loader = torch.utils.data.DataLoader(test_subset, **loader_kwargs)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

rm: cannot remove './data': No such file or directory


100%|██████████| 170498071/170498071 [00:04<00:00, 42462636.88it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


### Define CNN models and util methods
- (over)simpified CNN
- VGG11 (definition borrowed from *https://github.com/chengyangfu/pytorch-vgg-cifar10*)


In [3]:
class SimpleCNN(nn.Module):
    '''
    simple CNN model
    '''
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 80)
        self.fc3 = nn.Linear(80, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


class VGG(nn.Module):
    '''
    VGG model 
    '''
    def __init__(self, features):
        super(VGG, self).__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Linear(512, 10),
        )
        # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, np.sqrt(2. / n))
                m.bias.data.zero_()

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

vgg_cfg = {
    'vgg11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'vgg19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
    'vgg22': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 256, 'M', 512, 512, 512, 512, 512, 'M', 512, 512, 512, 512, 512, 'M']
}


def train_with_regularization(model, device, train_loader, optimizer, criterion, 
                              epoch_number, 
                              L1_lambda, 
                              L2_lambda):
    model.train()
    train_loss = 0.
    # get subsequent batches over the data in a given epoch
    for batch_idx, (data, target) in enumerate(train_loader):
        # send data tensors to GPU (or CPU)
        data, target = data.to(device), target.to(device)
        # this will zero out the gradients for this batch
        optimizer.zero_grad()
        # this will execute the forward() function
        output = model(data)
        # calculate loss using c
        loss = criterion(output, target)
        # L2 regularization implemeted by hand
        L2_norm = sum((p**2).sum() for p in model.parameters())
        # L1 regularization
        L1_norm = sum(p.abs().sum() for p in model.parameters())
        #
        loss_regularized = loss + L1_norm*L1_lambda + L2_norm*L2_lambda
        # backpropagate the loss
        loss_regularized.backward()
        # update the model weights (with assumed learning rate)
        optimizer.step()
        train_loss += loss.item()
    print('Train Epoch: {}'.format(epoch_number))
    train_loss /= len(train_loader)
    print('\tTrain set: Average loss: {:.4f}'.format(train_loss))
    return train_loss
    
def test(model, device, test_loader, criterion, message=None):
    model.eval()
    test_loss = 0.
    correct = 0
    # this is just inference, we don't need to calculate gradients
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device) 
            output = model(data)
            # calculate and sum up batch loss
            test_loss += criterion(output, target) 
            # get the index of class with the max probability 
            prediction = output.argmax(dim=1)  
            #_, predicted = torch.max(outputs.data, axis=1)
            # item() returns value of the given tensor
            correct += prediction.eq(target).sum().item()
    test_loss /= len(test_loader)
    accuracy = correct / len(test_loader.dataset)
    if message is not None:
        print('\t{}: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
            message, test_loss, correct, len(test_loader.dataset), 100.*accuracy))
    return test_loss.cpu(), accuracy

def run_training(model, device, criterion, optimizer, no_epochs,
                 L1_lambda=0., 
                 L2_lambda=0.):
    train_loss = []
    validation_loss = []
    validation_accuracy = []
    test_accuracy = []
    for epoch_number in range(1, no_epochs+1):
        train_loss.append(train_with_regularization(model, device, train_loader, 
                                optimizer, criterion, epoch_number,
                                L1_lambda, L2_lambda))
        val_loss, val_acc = test(model, device, validation_loader, 
                                criterion, 'Validation set')
        validation_loss.append(val_loss)
        validation_accuracy.append(val_acc)
        # we also collect test accuracies for every epoch
        _, test_acc = test(model, device, test_loader, criterion)
        test_accuracy.append(test_acc)
    # and select test accuracy for the best epoch (with the highest validation accuracy)
    best_accuracy = test_accuracy[np.argmax(validation_accuracy)]
    return train_loss, validation_loss, best_accuracy

def plot_loss(train_loss, validation_loss, title):
    plt.grid(True)
    plt.xlabel("subsequent epochs")
    plt.ylabel('average loss')
    plt.plot(range(1, len(train_loss)+1), train_loss, 'o-', label='training')
    plt.plot(range(1, len(validation_loss)+1), validation_loss, 'o-', label='validation')
    plt.legend()
    plt.title(title)
    plt.show()

# 0. Try to play with L2 and L1 *lambdas* do get highest possible accuracy.
- e.g. what if we increase *L1_lambda* to be the same as *L2_lambda*

# nazwy plikow sie powtarzajac zmienic loss na loss

## -> there is a code for extracting weights below
## -> what shape is model_0_params after training

In [4]:
def plot_and_save_weights(model_params,name,bins=50):
  weights = np.concatenate([p.flatten() for p in model_params])
  np.savetxt(name+".txt",weights)
  # Plot histogram of weights
  plt.hist(weights, bins=bins)
  plt.title(name)
  plt.xlabel('Weight values')
  plt.ylabel('Frequency')
  plt.title('Histogram of model weights')

### Do the experiments
- without any regularization
- *weigt_decay*, L2 regularization implemeted in optimizer, e.g. Adam or SGD as here: see [discussion](https://discuss.pytorch.org/t/how-does-sgd-weight-decay-work/33105/6)
- L2 and L1 regularization implemeted by hand

Simple CNN Without

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), 
                      lr=model_args['lr'],
                      weight_decay=0.)

train_loss_0, val_loss_0, best_accuracy = run_training(model, device, criterion, optimizer, model_args['epochs'])

print('\nTest accuracy for best epoch: {:.0f}%\n'.format(100.*best_accuracy))
plot_loss(train_loss_0, val_loss_0, 'Simple CNN Without any regularization')
model_0_params = [p.detach().cpu().numpy() for p in model.parameters()]

Train Epoch: 1
	Train set: Average loss: 2.0801
	Validation set: Average loss: 1.9122, Accuracy: 1404/5000 (28%)
Train Epoch: 2
	Train set: Average loss: 1.7046
	Validation set: Average loss: 1.7341, Accuracy: 1933/5000 (39%)
Train Epoch: 3
	Train set: Average loss: 1.5329
	Validation set: Average loss: 1.5469, Accuracy: 2225/5000 (44%)
Train Epoch: 4
	Train set: Average loss: 1.4245
	Validation set: Average loss: 1.7246, Accuracy: 2054/5000 (41%)
Train Epoch: 5
	Train set: Average loss: 1.3388
	Validation set: Average loss: 1.3906, Accuracy: 2503/5000 (50%)
Train Epoch: 6
	Train set: Average loss: 1.2598
	Validation set: Average loss: 1.3228, Accuracy: 2613/5000 (52%)
Train Epoch: 7
	Train set: Average loss: 1.1810
	Validation set: Average loss: 1.3804, Accuracy: 2540/5000 (51%)
Train Epoch: 8
	Train set: Average loss: 1.1136
	Validation set: Average loss: 1.3467, Accuracy: 2684/5000 (54%)
Train Epoch: 9
	Train set: Average loss: 1.0565
	Validation set: Average loss: 1.3713, Accuracy:

In [ ]:
name = "Simple CNN without regularization"
np.savetxt(name+"_loss.txt",[train_loss_0,val_loss_0])
plot_and_save_weights(model_0_params,name)

VGG without

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = VGG(make_layers(vgg_cfg['vgg11'])).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), 
                      lr=model_args['lr'],
                      weight_decay=0.)

train_loss_0, val_loss_0, best_accuracy = run_training(model, device, criterion, optimizer, model_args['epochs'])

print('\nTest accuracy for best epoch: {:.0f}%\n'.format(100.*best_accuracy))
plot_loss(train_loss_0, val_loss_0, 'VGG11 Without any regularization')
model_VGG_0_params = [p.detach().cpu().numpy() for p in model.parameters()]

In [ ]:
name = "VGG11 without regularization"
np.savetxt(name+"_loss.txt",[train_loss_0,val_loss_0])
plot_and_save_weights(model_VGG_0_params,name)

Simple CNN weight decay

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), 
                      lr=model_args['lr'],
                      weight_decay=model_args['L2_lambda'])

train_loss_wd, val_loss_wd, best_accuracy = run_training(model, device, criterion, optimizer, model_args['epochs'])

print('\nTest accuracy for best epoch: {:.0f}%\n'.format(100.*best_accuracy))
plot_loss(train_loss_wd, val_loss_wd, 'Simple CNN SGD weight_decay used')
model_wd_params = [p.detach().cpu().numpy() for p in model.parameters()]

In [ ]:
name = "Simple CNN weight decay"
np.savetxt(name+"_loss.txt",[train_loss_wd,val_loss_wd])
plot_and_save_weights(model_wd_params,name)

VGG11 weight decay

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = VGG(make_layers(vgg_cfg['vgg11'])).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), 
                      lr=model_args['lr'],
                      weight_decay=model_args['L2_lambda'])

train_loss_wd, val_loss_wd, best_accuracy = run_training(model, device, criterion, optimizer, model_args['epochs'])

print('\nTest accuracy for best epoch: {:.0f}%\n'.format(100.*best_accuracy))
plot_loss(train_loss_wd, val_loss_wd, 'VGG11 SGD weight_decay used')
model_VGGwd_params = [p.detach().cpu().numpy() for p in model.parameters()]

In [ ]:
name = "VGG11 weight decay"
np.savetxt(name+"_loss.txt",[train_loss_wd,val_loss_wd])
plot_and_save_weights(model_VGGwd_params,name)

Simple CNN L2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), 
                      lr=model_args['lr'],
                      weight_decay=0.)

train_loss_l2, val_loss_l2, best_accuracy = run_training(model, device, criterion, optimizer, model_args['epochs'],
                                                         L1_lambda=0.,
                                                         L2_lambda=model_args['L2_lambda']/2.)

print('\nTest accuracy for best epoch: {:.0f}%\n'.format(100.*best_accuracy))
plot_loss(train_loss_l2, val_loss_l2, 'L2 implemented by hand')
model_l2_params = [p.detach().cpu().numpy() for p in model.parameters()]

In [ ]:
name = "Simple CNN L2"
np.savetxt(name+"_loss.txt",[train_loss_l2,val_loss_l2])
plot_and_save_weights(model_l2_params,name)

VGG11 L2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = VGG(make_layers(vgg_cfg['vgg11'])).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), 
                      lr=model_args['lr'],
                      weight_decay=0.)

train_loss_l2, val_loss_l2, best_accuracy = run_training(model, device, criterion, optimizer, model_args['epochs'],
                                                         L1_lambda=0.,
                                                         L2_lambda=model_args['L2_lambda']/2.)

print('\nTest accuracy for best epoch: {:.0f}%\n'.format(100.*best_accuracy))
plot_loss(train_loss_l2, val_loss_l2, 'VGG11 L2 implemented by hand')
model_VGGl2_params = [p.detach().cpu().numpy() for p in model.parameters()]

In [ ]:
name = "VGG11 L2"
np.savetxt(name+"_loss.txt",[train_loss_l2,val_loss_l2])
plot_and_save_weights(model_VGGl2_params,name)

Simple CNN L2_lambda=L1_lambda

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), 
                      lr=model_args['lr'],
                      weight_decay=0.)

train_loss_l2, val_loss_l2, best_accuracy = run_training(model, device, criterion, optimizer, model_args['epochs'],
                                                         L1_lambda=model_args['L2_lambda']/2.,
                                                         L2_lambda=model_args['L2_lambda']/2.)

print('\nTest accuracy for best epoch: {:.0f}%\n'.format(100.*best_accuracy))
plot_loss(train_loss_l2, val_loss_l2, 'Simple CNN L2_lambda=L1_lambda')
model_l1l2_params = [p.detach().cpu().numpy() for p in model.parameters()]

In [ ]:
name = "Simple CNN L2 L1"
np.savetxt(name+"_loss.txt",[train_loss_l2,val_loss_l2])
plot_and_save_weights(model_l1l2_params,name)

VGG11 L2_lambda=L1_lambda

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = VGG(make_layers(vgg_cfg['vgg11'])).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), 
                      lr=model_args['lr'],
                      weight_decay=0.)

train_loss_l1, val_loss_l1, best_accuracy = run_training(model, device, criterion, optimizer, model_args['epochs'],
                                                         L1_lambda=model_args['L2_lambda']/2.,
                                                         L2_lambda=model_args['L2_lambda']/2.)

print('\nTest accuracy for best epoch: {:.0f}%\n'.format(100.*best_accuracy))
plot_loss(train_loss_l1, val_loss_l1, 'L1 implemented by hand')
model_vggl1l2_params = [p.detach().cpu().numpy() for p in model.parameters()]

In [ ]:
name = "VGG11 L2 L1"
np.savetxt(name+"_loss.txt",[train_loss_l1,val_loss_l1])
plot_and_save_weights(model_vggl1l2_params,name)

Simple CNN L1

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), 
                      lr=model_args['lr'],
                      weight_decay=0.)

train_loss_l1, val_loss_l1, best_accuracy = run_training(model, device, criterion, optimizer, model_args['epochs'],
                                                         L1_lambda=model_args['L1_lambda'],
                                                         L2_lambda=0.)

print('\nTest accuracy for best epoch: {:.0f}%\n'.format(100.*best_accuracy))
plot_loss(train_loss_l1, val_loss_l1, 'L1 implemented by hand')
model_l1_params = [p.detach().cpu().numpy() for p in model.parameters()]

In [ ]:
name = "Simple CNN L1"
np.savetxt(name+"_loss.txt",[train_loss_l1,val_loss_l1])
plot_and_save_weights(model_l1_params,name)

VGG11 L1

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = VGG(make_layers(vgg_cfg['vgg11'])).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), 
                      lr=model_args['lr'],
                      weight_decay=0.)

train_loss_l1, val_loss_l1, best_accuracy = run_training(model, device, criterion, optimizer, model_args['epochs'],
                                                         L1_lambda=model_args['L1_lambda'],
                                                         L2_lambda=0.)

print('\nTest accuracy for best epoch: {:.0f}%\n'.format(100.*best_accuracy))
plot_loss(train_loss_l1, val_loss_l1, 'VGG, L1 implemented by hand')
model_vggl1_params = [p.detach().cpu().numpy() for p in model.parameters()]

In [ ]:
name = "VGG11 L1"
np.savetxt(name+"_loss.txt",[train_loss_l1,val_loss_l1])
plot_and_save_weights(model_vggl1_params,name)

In [ ]:
plt.rcParams.update({'font.size': 12})

plt.grid(True)
plt.xlabel("subsequent epochs")
plt.ylabel('average loss')
range_epochs = model_args['epochs']+1
plt.plot(range(1, range_epochs), train_loss_0,  'o-', label='train_vanilla')
plt.plot(range(1, range_epochs), train_loss_wd, 'o-', label='train_weight_decay')
plt.plot(range(1, range_epochs), train_loss_l2, 'o-', label='train_L2')
plt.plot(range(1, range_epochs), train_loss_l1, 'o-', label='train_L1')
plt.legend()
plt.title("regularizers comparison - training")
plt.show()

plt.grid(True)
plt.xlabel("subsequent epochs")
plt.ylabel('average loss')
range_epochs = model_args['epochs']+1
plt.plot(range(1, range_epochs), val_loss_0,  'o-', label='validation_vanilla')
plt.plot(range(1, range_epochs), val_loss_wd, 'o-', label='validation_weight_decay')
plt.plot(range(1, range_epochs), val_loss_l2, 'o-', label='validation_L2')
plt.plot(range(1, range_epochs), val_loss_l1, 'o-', label='validation_L1')
plt.legend()
plt.title("regularizers comparison - validation")
plt.show()

Example weights:

In [ ]:
for p in model_0_params:
    print(p.shape)

np.set_printoptions(formatter={'float_kind': "{:.3f}".format})
# parameters for the second from last layer
print("vanilla:\n", model_0_params[-2][0])
print("L2 regularized:\n", model_l2_params[-2][0])
print("L1 regularized:\n", model_l1_params[-2][0])

(6, 3, 5, 5)
(6,)
(16, 6, 5, 5)
(16,)
(120, 400)
(120,)
(80, 120)
(80,)
(10, 80)
(10,)
vanilla:
 [0.283 0.005 -0.117 -0.248 0.044 0.295 -0.048 -0.100 0.252 0.109 -0.008
 0.149 0.026 0.128 0.004 0.098 -0.101 0.025 0.006 0.230 0.145 0.128 0.123
 -0.204 -0.316 -0.208 -0.111 -0.234 0.152 -0.308 -0.002 0.358 -0.109 0.164
 0.044 0.214 0.051 -0.282 0.046 -0.195 -0.191 0.280 0.192 0.194 0.105
 -0.312 0.030 0.151 -0.010 0.123 0.195 -0.281 -0.251 0.121 -0.044 0.197
 0.085 -0.031 0.204 -0.192 -0.135 -0.224 0.218 -0.283 -0.432 -0.262 0.173
 -0.146 0.199 -0.310 -0.020 0.166 0.030 0.012 0.121 0.361 0.048 0.327
 -0.152 0.181]
L2 regularized:
 [0.020 0.124 0.055 -0.155 -0.034 0.275 0.161 -0.170 0.038 -0.009 0.174
 0.036 0.148 -0.091 0.211 -0.247 -0.002 -0.082 -0.214 0.087 0.012 -0.155
 0.202 0.147 -0.172 0.080 0.038 0.083 0.005 0.214 0.012 -0.073 0.072 0.018
 -0.025 0.116 0.002 -0.005 -0.111 -0.027 0.169 -0.161 -0.148 0.164 -0.034
 0.077 -0.114 0.032 -0.227 0.364 0.013 -0.098 0.273 0.184 0.035 -0.089


### Tasks to do:

0. Try to play with L2 and L1 *lambdas* do get highest possible accuracy.
- e.g. what if we increase *L1_lambda* to be the same as *L2_lambda*
1. Repeat the calculations but for the VGG11 model.
2. Compare amplitudes (plot histograms?) of model weights for the L2 and L1 case -- is L1 a strong feature selector?

3. Reimplement pytorch [SGD method](https://pytorch.org/docs/stable/_modules/torch/optim/sgd.html#SGD) to get L1 regularization (|w| instead of w**2)

Hint: change line
```python
if weight_decay != 0:
    d_p = d_p.add(param, alpha=weight_decay)
```
to
```python
if weight_decay != 0:
    d_p = d_p.add(torch.sign(param), alpha=weight_decay) 
```